In [1]:
import os

os.chdir("..")

In [2]:
from neuralforecast.models import NBEATS
from neuralforecast import NeuralForecast
from utils import load_dataset, train_test_split
from models.NBeatsMoe import NBeatsMoe, NBEATSMoEBlock
import torch.nn as nn
import pandas as pd

# datasets list
datasets = [
    {
        "name": "gluonts_tourism_monthly"
    },
    {
        "name": "m3",
        "directory": "/home/ricardo/mixture_of_experts_time_series/data/m3/",
        "group": "Monthly",
        "freq": "M",
    },
    {
        "name": "m4",
        "directory": "/home/ricardo/mixture_of_experts_time_series/data/m4/",
        "group": "Monthly",
        "freq": "M",
    } 
]

c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-29 16:59:04,267	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-29 16:59:04,443	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\gluonts\json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [3]:
Y_ALL_m4 = load_dataset("m4", datasets[2])
Y_ALL_m3 = load_dataset("m3", datasets[1])
n_lags = 36
horizon = 18

Loading m4_monthly dataset...
Loading m3_monthly dataset...


In [4]:
import pandas as pd

start_date = pd.to_datetime('1994-01-31')

def convert_to_date(group):
    group = group.copy()
    periods = len(group)
    group['unique_id'] = "4" + group['unique_id']
    group['ds'] = pd.date_range(start=start_date, periods=periods, freq='M')
    return group

Y_ALL_m4 = Y_ALL_m4.groupby('unique_id', group_keys=False).apply(convert_to_date)
Y_ALL_m4

,unique_id,ds,y
0,4M1,1994-01-31,8000.0
1,4M1,1994-02-28,8350.0
2,4M1,1994-03-31,8570.0
3,4M1,1994-04-30,7700.0
4,4M1,1994-05-31,7080.0
...,...,...,...
11246406,4M9999,2000-11-30,4200.0
11246407,4M9999,2000-12-31,4300.0
11246408,4M9999,2001-01-31,3800.0
11246409,4M9999,2001-02-28,4400.0


In [5]:
print(n_lags)
Y_train_df_m4, Y_test_df_m4 = train_test_split(Y_ALL_m4, horizon)
Y_train_df_m3, Y_test_df_m3 = train_test_split(Y_ALL_m3, horizon)

36


In [6]:
models_m4 = NBEATS(h=horizon, input_size=n_lags, max_steps=1500)
models_m3 = NBEATS(h=horizon, input_size=n_lags)

Seed set to 1
Seed set to 1


In [7]:
fcst_m4 = NeuralForecast(models=[models_m4], freq="M")
fcst_m4.fit(df=Y_train_df_m4, static_df=None, val_size=horizon)

fcst_m3 = NeuralForecast(models=[models_m3], freq="M")
fcst_m3.fit(df=Y_train_df_m3, static_df=None, val_size=horizon)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.5 M  | train
-------------------------------------------------------
2.5 M     Trainable params
2.0 K     Non-trainable params
2.5 M     Total params
9.954     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 1500/1500 [06:53<00:00,  3.63it/s, v_num=124, train_loss_step=444.0, valid_loss=506.0, train_loss_epoch=389.0]

`Trainer.fit` stopped: `max_steps=1500` reached.


Epoch 0: 100%|██████████| 1500/1500 [06:53<00:00,  3.63it/s, v_num=124, train_loss_step=444.0, valid_loss=506.0, train_loss_epoch=389.0]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.5 M  | train
-------------------------------------------------------
2.5 M     Trainable params
2.0 K     Non-trainable params
2.5 M     Total params
9.954     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 22:  22%|██▏       | 10/45 [00:01<00:04,  8.10it/s, v_num=125, train_loss_step=450.0, train_loss_epoch=561.0, valid_loss=661.0]

`Trainer.fit` stopped: `max_steps=1000` reached.


Epoch 22:  22%|██▏       | 10/45 [00:01<00:04,  8.08it/s, v_num=125, train_loss_step=450.0, train_loss_epoch=561.0, valid_loss=661.0]


Now lets add the 2 datasets together ...

In [8]:
blcs_m4 = fcst_m4.models[0].blocks
blcs_m3 = fcst_m3.models[0].blocks

... and the 2 blocks

In [9]:
added_layers = nn.ModuleList()
for i in zip(blcs_m4, blcs_m3):
    blc1, blc2 = i
    n_theta = blc1.layers[-1].out_features

    #freeze training layers
    for param in blc1.layers.parameters():
        param.requires_grad = False
    for param in blc2.layers.parameters():
        param.requires_grad = False

    added_layers.append(NBEATSMoEBlock(
        input_size=n_lags,
        top_k=1,
        nr_experts=2,
        n_theta=n_theta,
        mlp_units=[[16,16], [16,16], [16,16]], ## just mocked
        basis=blc1.basis,
        activation="ReLU",
        dropout_prob=blc1.dropout_prob,
        pre_experts=nn.ModuleList([blc1.layers, blc2.layers]),
    ))


In [10]:
# moe = NBeatsMoe(
#     pre_blocks=added_layers,
#     nr_experts=2,
#     top_k=1, # TODO: try changing this to 2
#     input_size=n_lags,
#     h=horizon,
#     learning_rate=1e-5,
#     max_steps=1500,
#     gate_type="conv1d-aap",
# )
nbeats = NBEATS(
    mlp_units= 3 * [[128 * 2, 128 * 2]],
    input_size=n_lags,
    h=horizon,
    max_steps=1500,
)

Seed set to 1


Let's train ...

In [11]:
## add both y_all
Y_ALL = Y_ALL_m4.copy()
Y_ALL = pd.concat([Y_ALL, Y_ALL_m3], ignore_index=True)

# split train and test
Y_train_df, Y_test_df = train_test_split(Y_ALL, horizon)

In [12]:
fcts_tl = NeuralForecast(models=[nbeats], freq="M")
fcts_tl.fit(df=Y_train_df, static_df=None, val_size=horizon)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 655 K  | train
-------------------------------------------------------
653 K     Trainable params
2.0 K     Non-trainable params
655 K     Total params
2.622     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 0:  97%|█████████▋| 1500/1545 [04:45<00:08,  5.25it/s, v_num=136, train_loss_step=278.0, valid_loss=512.0, train_loss_epoch=391.0]

`Trainer.fit` stopped: `max_steps=1500` reached.


Epoch 0:  97%|█████████▋| 1500/1545 [04:45<00:08,  5.25it/s, v_num=136, train_loss_step=278.0, valid_loss=512.0, train_loss_epoch=391.0]


In [13]:
# lets predict ...
predictions = fcts_tl.predict(futr_df=Y_test_df)
predictions_m4 = fcst_m4.predict(futr_df=Y_test_df_m4)
predictions_m3 = fcst_m3.predict(futr_df=Y_test_df_m3)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1545/1545 [00:12<00:00, 125.57it/s]


NameError: name 'fcst_m4' is not defined

In [18]:
from neuralforecast.losses.numpy import smape

# Calculate SMAPE for each prediction

# For predictions from fcts_tl
def calculate_smape(Y_test, predictions, model_name):
    y_true = Y_test['y'].values
    y_hat = predictions[model_name].values

    n_series = Y_test['unique_id'].nunique()
    y_true = y_true.reshape(n_series, -1)
    y_hat = y_hat.reshape(n_series, -1)

    smape_value = smape(y_true, y_hat)
    return smape_value

# Calculate SMAPE for fcts_tl
smape_tl = calculate_smape(Y_test_df, predictions, moe.__class__.__name__)
smape_m4 = calculate_smape(Y_test_df_m4, predictions_m4, "NBEATS")
smape_m3 = calculate_smape(Y_test_df_m3, predictions_m3, "NBEATS")

# Print SMAPE values
print(f"SMAPE for fcts_tl: {smape_tl}")
print(f"SMAPE for fcst_m4: {smape_m4}")
print(f"SMAPE for fcst_m3: {smape_m3}")

NameError: name 'moe' is not defined

In [ ]:
Y_test_df_m3 # TODO: change the name of the unique ids for the m3 or m4

,unique_id,ds,y
0,M1,1994-03-31,2280.0
1,M1,1994-04-30,480.0
2,M1,1994-05-31,5040.0
3,M1,1994-06-30,1920.0
4,M1,1994-07-31,840.0
...,...,...,...
25699,M999,1993-10-31,5225.9
25700,M999,1993-11-30,5236.3
25701,M999,1993-12-31,5186.6
25702,M999,1994-01-31,5143.4


Test the moe on both datasets independently ....

In [ ]:
predictions_tl_m4 = predictions[~predictions['unique_id'].str.match(r'^M\d+')]
predictions_tl_m3 = predictions[predictions['unique_id'].str.match(r'^M\d+')]

# Calculate SMAPE for fcts_tl
smape_tl_m4 = calculate_smape(Y_test_df_m4, predictions_tl_m4, moe.__class__.__name__)
smape_tl_m3 = calculate_smape(Y_test_df_m3, predictions_tl_m3, moe.__class__.__name__)

# Print SMAPE values
print(f"SMAPE for fcts_tl_m4: {smape_tl_m4}")
print(f"SMAPE for fcts_tl_m3: {smape_tl_m3}")

SMAPE for fcts_tl_m4: 0.13296226673629807
SMAPE for fcts_tl_m3: 0.14294176055413135


predict on other dataset ... 


In [14]:
Y_ALL_tourism = load_dataset("gluonts_tourism_monthly", datasets[0])[0]

INFO:root:using dataset already processed in path C:\Users\ricar\.gluonts\datasets\tourism_monthly.


Loading tourism_monthly dataset...


In [15]:
Y_train_df_tourism, Y_test_df_tourism = train_test_split(Y_ALL_tourism, horizon)

In [16]:
predictions_tourism = fcts_tl.predict(df=Y_train_df_tourism,futr_df=Y_test_df_tourism)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 85.78it/s]


In [20]:
smape_tl_tourism = calculate_smape(Y_test_df_tourism, predictions_tourism, nbeats.__class__.__name__)

print(f"SMAPE for fcts_tl_tourism: {smape_tl_tourism}")

SMAPE for fcts_tl_tourism: 0.26391877672528485
